In [1]:
import pandas as pd
import numpy as np
import datetime
from yahoo_fin.stock_info import get_data,get_quote_table
import statsmodels.formula.api as smf
from pypfopt.efficient_frontier import EfficientFrontier

import json
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

In [2]:
CC_string = "ADA,ADX,AE,ANT,ARDR,ARK,BAT,BCH,BCN,BNB,BNT,BTC,BTG,BTS,CVC,DASH,DCR,DGB,DGD,DNT,DOGE,EDG,EOS,ETC,ETH,FUN,GAS,GBYTE,GNO,HC,ICX,KIN,KMD,KNC,LINK,LRC,LSK,LTC,MAID,MCO,MIOTA,MLN,MONA,MTL,NANO,NEO,NXS,OMG,PIVX,PPT,QRL,QTUM,REP,RLC,SALT,SC,SNT,STEEM,STORJ,SYS,TRX,VERI,VET,WAVES,WTC,XEM,XLM,XMR,XRP,XVG,ZEC,ZEN,ZRX"
CC_temp_list = CC_string.split(",")
CC_list = [i+'-USD' for i in CC_temp_list]

In [3]:
#Output the top 10 CC
def top10MarketCapCC():    
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest?convert=USD'
    parameters = {
        'symbol' : 'ADA,ADX,AE,ANT,ARDR,ARK,BAT,BCH,BCN,BNB,BNT,BTC,BTG,BTS,CVC,DASH,DCR,DGB,DGD,DNT,DOGE,EDG,EOS,ETC,ETH,FUN,GAS,GBYTE,GNO,HC,ICX,KIN,KMD,KNC,LINK,LRC,LSK,LTC,MAID,MCO,MIOTA,MLN,MONA,MTL,NANO,NEO,NXS,OMG,PIVX,PPT,QRL,QTUM,REP,RLC,SALT,SC,SNT,STEEM,STORJ,SYS,TRX,VERI,VET,WAVES,WTC,XEM,XLM,XMR,XRP,XVG,ZEC,ZEN,ZRX'
    }
    headers = {
        'Accepts': 'application/json',
    #Input your API key here
        'X-CMC_PRO_API_KEY': '4797f5ee-9a2d-42d1-979a-ae14643c966d',
    }
    session = Session()
    session.headers.update(headers)
    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        result = data
        d = pd.DataFrame(result)
        f = d.iloc[6:,1]
        MarketCap_rank = pd.DataFrame(columns = ['Symbol', 'MarketCap'])
        for i in range(73):
            Append = {'Symbol': f.iloc[i]['symbol'], 'MarketCap' : f.iloc[i]['quote']['USD']['market_cap']}
            MarketCap_rank = MarketCap_rank.append(Append, ignore_index=True)
        MarketCap_rank = MarketCap_rank.sort_values(by=['MarketCap'], ascending = False)
        MarketCap_rank.reset_index(drop=True, inplace=True)
        MarketCap_rank = MarketCap_rank.truncate(before=None, after=9, copy=True)
        return MarketCap_rank#['Symbol'] + '-USD'
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)

In [4]:
top10MarketCapCC_list = top10MarketCapCC()

In [5]:
top10MarketCapCC_list

,Symbol,MarketCap
0,BTC,6.274345e+11
1,ETH,1.539933e+11
2,XRP,1.330385e+10
3,ADA,1.063334e+10
4,LINK,9.114588e+09
5,LTC,8.709520e+09
6,BCH,7.423747e+09
7,BNB,6.677035e+09
8,XLM,6.473797e+09
9,DOGE,3.953708e+09


In [6]:
weekly_data = get_data("ADA-USD", interval = "1wk")

In [7]:
weekly_data

,open,high,low,close,adjclose,volume,ticker
2017-09-24,0.021678,0.032226,0.017354,0.024969,0.024969,50068700,ADA-USD
2017-10-01,0.024607,0.030088,0.017620,0.020477,0.020477,112121070,ADA-USD
2017-10-08,0.020344,0.038268,0.020342,0.030049,0.030049,70248060,ADA-USD
2017-10-15,0.029939,0.030855,0.024637,0.027729,0.027729,35460970,ADA-USD
2017-10-22,0.027684,0.029837,0.023766,0.028607,0.028607,25102830,ADA-USD
...,...,...,...,...,...,...,...
2021-01-04,0.205236,0.353344,0.194450,0.304895,0.304895,30627294564,ADA-USD
2021-01-11,0.304996,0.395653,0.233033,0.379852,0.379852,27415450604,ADA-USD
2021-01-18,0.379735,0.395443,0.283576,0.353881,0.353881,22674159877,ADA-USD
2021-01-25,0.353850,0.366772,0.305461,0.348131,0.348131,13911430300,ADA-USD


In [8]:
#Output market returns
def MarketReturns(CC_list):
    data = pd.DataFrame()
    end = datetime.date.today()
    start = datetime.datetime.now() - datetime.timedelta(days = 7 * 8)
    for i in CC_list:
        data[i] = get_data(i, start, end,True, '1d')['adjclose']
    data= data.fillna(value=.001)#Usd because faulty API returns NaN values
    marketData = np.log(data/ data.shift(1)).fillna(value=0.001)
    sum_row = marketData.sum(axis=1)
    return sum_row
    

In [9]:
I = MarketReturns(CC_list)

KeyboardInterrupt: 

In [ ]:
J = pd.DataFrame(I, columns = ['MarketReturns'])

In [ ]:
#Load the list of CC used in the research
end = datetime.date.today()
start = datetime.datetime.now() - datetime.timedelta(days = 7 * 8)

data = pd.DataFrame()
for i in top10MarketCapCC_list:
    data[i] = get_data(i, start, end,True, '1d')['adjclose']
data= data.fillna(value=.001)
marketData = np.log(data/ data.shift(1)).fillna(value=0.001)

In [ ]:
result = pd.concat([marketData.rename(columns=lambda s: s.replace('-USD','')), J], axis=1)

In [ ]:
result.mean()
result.std()

In [ ]:
Name = list(result.iloc[:,0:10])
regression = pd.DataFrame(columns=['beta', 'rse'])
for i in range(10):
    CAPM_model = smf.ols(formula = Name[i]+' ~ MarketReturns', data = result)
    CAPM_fit = CAPM_model.fit()
    beta = CAPM_fit.params['MarketReturns']
    rse = CAPM_fit.bse['MarketReturns']
    dict = {'beta': beta, 'rse': rse} 
    df = pd.DataFrame(dict, index = [Name[i]])
    regression = regression.append(df)
    
print(regression)

In [ ]:
def calculateReturns(x):
    for i in range(10):
        y = str(x[i:i+1]).split(' ')[0]
        x[i] = result.mean()['MarketReturns'] * regression.loc[y]['beta']
    return x

In [ ]:
ExpectedReturns = pd.DataFrame(index = ['returns'],columns = list(result.iloc[:,0:10]))
ExpectedReturns = ExpectedReturns.apply(lambda x: calculateReturns(x), axis=1) 
ExpectedReturns.T

In [ ]:
def calculateCovariance(x):
    for i in range(10):
        y = str(x[i:i+1]).split(' ')[0]
        if(y == x.name):
            x[i] = (result.std()['MarketReturns'] * regression.loc[y]['beta']) ** 2 + regression.loc[y]['rse']
        else:
            x[i] = (result.std()['MarketReturns'] ** 2) * (regression.loc[y]['beta'] * regression.loc[x.name]['beta'])
    return x

In [ ]:
CovarianceMatrix = pd.DataFrame(index = list(result.iloc[:,0:10]),columns = list(result.iloc[:,0:10]))
CovarianceMatrix = CovarianceMatrix.apply(lambda x: calculateCovariance(x), axis=1) 
CovarianceMatrix 

In [ ]:
ef1 = EfficientFrontier(ExpectedReturns.iloc[0], CovarianceMatrix, weight_bounds=(0, 1))
weights = ef1.max_sharpe(risk_free_rate= 0.0)
weights = ef1.clean_weights()

weights